# ENCODING

In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
# Gráficos
# ==============================================================================
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Asunciones y Preprocesamiento
# ==============================================================================
from scipy import stats
import math
from scipy.stats import levene
import researchpy as rp
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import itertools
# ANOVA
# ==============================================================================
import statsmodels.api as sm
from statsmodels.formula.api import ols
#Linear Regresion con Sklearn
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
# Tamaño gráficas
# ==============================================================================
plt.rcParams["figure.figsize"] = (10,8)

In [2]:
df = pd.read_pickle("datos/bikes_clean_pickle.pkl")
df.head()

,date,season,year,month,holiday,weekday,workingday,weather,temp,hum,windspeed,casual,registered
0,2018-01-01,winter,0,1,holiday,Monday,weekend or holiday,cloudy,14.110847,80.5833,10.749882,331,654
1,2018-01-02,winter,0,1,not holiday,Tuesday,workingday,cloudy,14.902598,69.6087,16.652113,131,670
2,2018-01-03,winter,0,1,not holiday,Wednesday,workingday,clear,8.050924,43.7273,16.636703,120,1229
3,2018-01-04,winter,0,1,not holiday,Thursday,workingday,clear,8.200000,59.0435,10.739832,108,1454
4,2018-01-05,winter,0,1,not holiday,Friday,workingday,clear,9.305237,43.6957,12.522300,82,1518


## Para el map de workingday. 

In [3]:
estad = df.describe().T
estad

,count,mean,std,min,25%,50%,75%,max
year,730.0,0.500000,0.500343,0.000000,0.000000,0.500000,1.000000,1.000000
month,730.0,6.526027,3.450215,1.000000,4.000000,7.000000,10.000000,12.000000
temp,730.0,20.319259,7.506729,2.424346,13.811885,20.465826,26.880615,35.328347
hum,730.0,62.765175,14.237589,0.000000,52.000000,62.625000,72.989575,97.250000
windspeed,730.0,12.763620,5.195841,1.500244,9.041650,12.125325,15.625589,34.000021
casual,730.0,849.249315,686.479875,2.000000,316.250000,717.000000,1096.500000,3410.000000
registered,730.0,3658.757534,1559.758728,20.000000,2502.250000,3664.500000,4783.250000,6946.000000


In [4]:
df_workingday = df.groupby('workingday')['casual', 'registered'].median()
df_workingday

,casual,registered
workingday,,
weekend or holiday,735.5,3720.5
workingday,711.5,3633.0


Haciendo la proporción entre las medias de las categorías

In [5]:
mapa_wd_casual = {False: 1, True: (df_workingday.iloc[1,0] / df_workingday.iloc[0,0])}
mapa_wd_registered = {False: (df_workingday.iloc[0,1] / df_workingday.iloc[1,1]), True: 1}
print(f'Los valores del mapa de workingday para el modelo de casual sería: {mapa_wd_casual}')
print(f'Los valores del mapa de workingday para el modelo de casual sería: {mapa_wd_registered}')

Los valores del mapa de workingday para el modelo de casual sería: {False: 1, True: 0.9673691366417403}
Los valores del mapa de workingday para el modelo de casual sería: {False: 1.0240847784200386, True: 1}


Considerando la media global para sacar las proporciones

In [6]:
# si ajustamos en base a las medias de cada categoría de 'workingday' respecto a la media global de esa columna paralos casos de 'casual' y 'registered':
mapa_wd_casual = {False: (df_workingday.iloc[0,0] / estad.loc['casual', '50%']), True: (df_workingday.iloc[1,0] / estad.loc['casual', '50%'])}
mapa_wd_registered = {False: (df_workingday.iloc[0,1] / estad.loc['registered', '50%']), True: (df_workingday.iloc[1,1] / estad.loc['registered', '50%'])}
print(f'Los valores del mapa de workingday para el modelo de casual sería: {mapa_wd_casual}')
print(f'Los valores del mapa de workingday para el modelo de casual sería: {mapa_wd_registered}')

Los valores del mapa de workingday para el modelo de casual sería: {False: 1.0258019525801954, True: 0.9923291492329149}
Los valores del mapa de workingday para el modelo de casual sería: {False: 1.0152817574021011, True: 0.9914040114613181}
